In [1]:
import pandas as pd

### Data Prep

In [37]:
# SAT titles spread across 4 rows of uneven merges
raw_cols = pd.read_excel('https://nces.ed.gov/programs/digest/d18/tables/xls/tabn226.40.xls',
                   header=None, index_col = 0, skiprows = 1, nrows=4)
# Keep raw data separate
sat = pd.read_excel('https://nces.ed.gov/programs/digest/d18/tables/xls/tabn226.40.xls',
                   header=None, index_col = 0, skiprows=7)

# Manually relabel columns
sat.columns = ['sat_2017', 'sat_2017_sd',
               'sat_2017_erw', 'sat_2017_erw_sd',
               'sat_2017_math', 'sat_2017_math_sd',
               'sat_2017_part',
               'sat_2018', 'sat_2018_sd',
               'sat_2018_erw', 'sat_2018_erw_sd',
               'sat_2018_math', 'sat_2018_math_sd',
               'sat_2018_part']

# Remove 3 Footnotes
sat = sat.iloc[:-3,:]
# Remove blank lines
sat = sat.loc[~sat['sat_2017_erw'].isnull(),:]
sat.drop(columns = sat.columns[sat.columns.str.contains('sd')], inplace = True)

# Correct index
sat.index = sat.index.str.strip('.')
sat.index = sat.index.str.replace('Columbia', 'DC')
sat.index = sat.index.str.strip()

for col in sat.columns:
    sat[col] = sat[col].astype(int)

In [44]:
# ACT titles spread across 3 rows of uneven merges
raw_cols = pd.read_excel('https://nces.ed.gov/programs/digest/d18/tables/xls/tabn226.60.xls',
                   header=None, index_col = 0, skiprows = 1, nrows=3)

act_18 = pd.read_excel('https://nces.ed.gov/programs/digest/d18/tables/xls/tabn226.60.xls',
                   header=None, index_col = 0, skiprows=6)

# drop 3 Footnotes and 2014 data
act_18 = act_18.iloc[:-3,5:]

act_18.columns = ['act_2018', 'act_2018_english', 
                  'act_2018_math', 'act_2018_reading', 
                  'act_2018_science',
                  'act_2014_part', 'act_2018_part']

act_18.drop(columns=['act_2014_part'], inplace=True)
act_18 = act_18.loc[~act_18['act_2018'].isnull(),:]

# Correct index
act_18.index = act_18.index.str.strip('.')
act_18.index = act_18.index.str.replace('District of Columbia', 'DC')

for col in act_18.columns:
    act_18[col] = act_18[col].astype(int)



act_17 = pd.read_excel('https://nces.ed.gov/programs/digest/d17/tables/xls/tabn226.60.xls',
                   header=None, index_col = 0, skiprows=6)

# drop 3 Footnotes and 2014 data
act_17 = act_17.iloc[:-3,5:]

act_17.columns = ['act_2017', 'act_2017_english', 
                  'act_2017_math', 'act_2017_reading', 
                  'act_2017_science',
                  'act_2014_part', 'act_2017_part']

act_17.drop(columns=['act_2014_part'], inplace=True)
act_17 = act_17.loc[~act_17['act_2017'].isnull(),:]

# Correct index
act_17.index = act_17.index.str.strip('.')
act_17.index = act_17.index.str.replace('District of Columbia', 'DC')


for col in act_17.columns:
    act_17[col] = act_17[col].astype(int)
    
act = pd.merge(act_17, act_18, left_index=True, right_index=True)
act.index = act.index.str.strip()

In [48]:
# Create final panel dataset
test_scores = pd.merge(sat, act, how='outer', left_index=True, right_index=True)
test_scores.to_csv('../data/test_scores.csv')

### QC Checks

In [128]:
# Collections of columns of similar category for easy reference throughout 
sat_cols = test_scores.columns[test_scores.columns.str.contains('sat') &
                              ~test_scores.columns.str.contains('part') ]
sat_st_cols = sat_cols[sat_cols.str.contains('erw') | 
                       sat_cols.str.contains('math')]

act_cols = test_scores.columns[test_scores.columns.str.contains('act') &
                           ~test_scores.columns.str.contains('part') ]
part_cols = test_scores.columns[test_scores.columns.str.contains('part')]

In [129]:
# Return missing values count
print(f"Datasets contains {test_scores.isnull().sum().sum()} missing values.")


# Convert SAT to numeric and check range
for col in sat_st_cols:
    if test_scores[col].max() > 800:
        print(f"Error: {col} out of valid range.")
    elif test_scores[col].min()  < 200:
        print(f"Error: {col} out of valid range.")
        
for col in ['sat_2017', 'sat_2018']:
    if test_scores[col].max() > 1600:
        print(f"Error: {col} out of valid range.")
    elif test_scores[col].min() < 400:
        print(f"Error: {col} out of valid range.")
        
# Convert ACT to numeric and check range
for col in act_cols:
    if test_scores[col].max()  > 36:
        print(f"Error: {col} out of valid range.")
    elif test_scores[col].min()  < 1:
        print(f"Error: {col} out of valid range.")

# Convert participation to numeric and check range
for col in part_cols:
    if test_scores[col].max()  > 100:
        print(f"Error: {col} out of valid range.")
    elif test_scores[col].min()  < 0:
        print(f"Error: {col} out of valid range.")
        

Datasets contains 0 missing values.
